In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
import gmsh
import scipy
import numpy as np

### Load precomputed integrals

In [ ]:
lagrange_linear_poisson_matrix = dill.load(
    open("../calculations/lagrange_linear_poisson_matrix", "rb")
)

lagrange_linear_poisson_right_M_values = dill.load(
    open("../calculations/lagrange_linear_poisson_right_M_values", "rb")
)

### Run triangulation

In [ ]:
gmsh.initialize()
gmsh.model.add("quad_mesh")

ms = 0.7
# Define points
point_1 = gmsh.model.geo.addPoint(0, 0, 0, ms)
point_2 = gmsh.model.geo.addPoint(1, 1, 0, ms)
point_3 = gmsh.model.geo.addPoint(-1, 1, 0, ms)
point_4 = gmsh.model.geo.addPoint(-1, -1, 0, ms)
point_5 = gmsh.model.geo.addPoint(1, -1, 0, ms)

# Define circles
arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# arc_1 = gmsh.model.geo.addLine(point_2, point_3)
# arc_2 = gmsh.model.geo.addLine(point_3, point_4)
# arc_3 = gmsh.model.geo.addLine(point_4, point_5)
# arc_4 = gmsh.model.geo.addLine(point_5, point_2)


# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()

ms = 6
# gmsh.model.mesh.setTransfiniteCurve(arc_1, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_2, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_3, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_4, ms)
# gmsh.model.mesh.setTransfiniteSurface(plane)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")

gmsh.option.setNumber(name="Mesh.Smoothing", value=0)
gmsh.model.mesh.generate(2)
gmsh.fltk.run()

In [ ]:
import gmsh

gmsh.initialize()

# Start a new model
gmsh.model.add('polygon')

# Define the vertices               ######
vertices = [(0, 0), (2, 0), (2, 1), (1/8, 1/8), (1, 2), (0, 2)]
points = [gmsh.model.geo.addPoint(x, y, 0, 0.1) for x, y in vertices]


line_1 = gmsh.model.geo.addLine(points[0], points[1])
line_2 = gmsh.model.geo.addLine(points[1], points[2])
line_3 = gmsh.model.geo.addLine(points[2], points[3])
line_4 = gmsh.model.geo.addLine(points[3], points[0])

# Define the curve loops and surfaces
curve_loop1 = gmsh.model.geo.addCurveLoop([
        line_1,
        line_2,
        line_3,
        line_4
])

surface1 = gmsh.model.geo.addPlaneSurface([curve_loop1])

line_5 = gmsh.model.geo.addLine(points[3], points[4])
line_6 = gmsh.model.geo.addLine(points[4], points[5])
line_7 = gmsh.model.geo.addLine(points[5], points[0])

curve_loop2 = gmsh.model.geo.addCurveLoop([
        line_5,
        line_6,
        line_7,
        -line_4,
])

surface2 = gmsh.model.geo.addPlaneSurface([curve_loop2])


num_nodes = 30

gmsh.model.geo.mesh.setTransfiniteCurve(line_1, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_2, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_3, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_4, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_5, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_6, num_nodes)
gmsh.model.geo.mesh.setTransfiniteCurve(line_7, num_nodes)


# Set transfinite surfaces
gmsh.model.geo.mesh.setTransfiniteSurface(surface1)
gmsh.model.geo.mesh.setTransfiniteSurface(surface2)

# Synchronize the model and generate 2D mesh
gmsh.model.geo.synchronize()

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [line_1, line_2, line_3, line_5, line_6, line_7], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [surface2, surface1], name="Surface")

gmsh.model.mesh.generate(2)

gmsh.fltk.run()


In [ ]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [ ]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
triangles = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 2][0]
triangles = triangles.reshape(-1, 3).astype(int)
triangles = triangles - 1

In [ ]:
edges = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 1][0]
edges = edges.reshape(-1, 2).astype(int)
edges = edges - 1

In [ ]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
matrix = np.zeros((points.shape[0], points.shape[0]))
matrix_M = np.zeros((points.shape[0], points.shape[0]))

for idx, element in enumerate(tqdm(triangles)):
    x1, x2, x3 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y1, y2, y3 = points[element[0], 1], points[element[1], 1], points[element[2], 1]

    FF = lagrange_linear_poisson_matrix(x1, y1, x2, y2, x3, y3)
    FF_M = lagrange_linear_poisson_right_M_values(x1, y1, x2, y2, x3, y3)

    for local_point in range(3):        
        for i in range(3):
            matrix[element[local_point], element[i]] += FF[local_point, i]                
            matrix_M[element[local_point], element[i]] += FF_M[local_point, i]
                            
for bidx in np.where(vertex_marker_is_boundary == 1)[0]:
    matrix[bidx, :] *= 0
    matrix[:, bidx] *= 0
    matrix[bidx, bidx] = 1
    
    matrix_M[bidx, :] *= 0
    matrix_M[:, bidx] *= 0
    matrix_M[bidx, bidx] = 1

In [ ]:
_x_ = points[:, 0]
_y_ = points[:, 1]

In [ ]:
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(A=matrix, M=matrix_M, k=matrix.shape[0]-1, which="BE")
eigenvectors = eigenvectors.T

In [ ]:
eigenvalues, eigenvectors = scipy.linalg.eigh(a=matrix, b=matrix_M)
eigenvectors = eigenvectors.T

In [ ]:
n_boundary = np.count_nonzero(vertex_marker_is_boundary)

In [ ]:
# plt.scatter(_x_, _y_, c=eigenvectors[n_boundary+23], cmap="magma")
# plt.axis("equal")
# plt.show()

In [ ]:
import plotly

eidx = n_boundary+8
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=_x_, 
                                y=_y_,
                                z=eigenvectors[eidx],
                                intensity=eigenvectors[eidx],
                                i = triangles[:, 0],
                                j = triangles[:, 1],
                                k = triangles[:, 2],
                                )
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()